<a href="https://colab.research.google.com/github/Piyush07Mishra/ASEP-PROJECT1/blob/main/Artisitc%20Qr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import qrcode
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from PIL import Image, ImageOps, ImageEnhance
import gradio as gr
import io
from pyzbar.pyzbar import decode

# --- QR Generation Module ---
def generate_qr(data, size=512):
    qr = qrcode.QRCode(
        version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=10, border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    return img.resize((size, size))

# --- Blending Module ---
def blend_images(qr_img, bg_img, mode):
    qr_mask = qr_img.convert("L").point(lambda x: 0 if x < 128 else 255, '1')
    bg_img = bg_img.resize(qr_img.size)
    if mode == "Overlay":
        composite = Image.composite(qr_img, bg_img, qr_mask)
    elif mode == "Multiply":
        composite = Image.blend(bg_img, qr_img, alpha=0.5)
    else:
        composite = Image.alpha_composite(bg_img.convert("RGBA"), qr_img.convert("RGBA"))
    return composite.convert("RGB")

# --- AI Model Setup ---
controlnet = ControlNetModel.from_pretrained(
    "monster-labs/control_v1p_sd15_qrcode_monster",
    torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# --- Scannability Check Module ---
def is_qr_scannable(image):
    image = image.convert("RGB")
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)
    result = decode(Image.open(buffer))
    return len(result) > 0

# --- Main Pipeline ---
def qr_stylizer(
    data, prompt, negative_prompt, background_mode, user_bg, blend_mode,
    guidance_scale, controlnet_scale, num_steps
):
    qr_img = generate_qr(data)

    # Background selection
    if background_mode == "AI Generation":
        bg_img = None
    else:
        if user_bg is None:
            return None, "Please upload a background image."
        bg_img = Image.open(user_bg).convert("RGB")
        qr_blended = blend_images(qr_img, bg_img, blend_mode)
    # Generate with ControlNet
    if background_mode == "AI Generation":
        base_img = qr_img
    else:
        base_img = qr_blended

    generated = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=base_img,
        guidance_scale=float(guidance_scale),
        controlnet_conditioning_scale=float(controlnet_scale),
        num_inference_steps=int(num_steps),
    ).images[0]

    # Validate QR
    scan_ok = is_qr_scannable(generated)
    if not scan_ok:
        msg = "Warning: QR code may not be scannable! Try increasing ControlNet scale, using less dense blends, or a simpler prompt."
    else:
        msg = "✅ Success: QR code is scannable."

    return generated, msg

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("## 🎨 Artistic, Scannable QR Code Generator")
    with gr.Row():
        with gr.Column():
            data = gr.Textbox(label="Data (URL/Text)", value="https://google.com")
            prompt = gr.Textbox(label="Prompt (For AI Generation)", value="Futuristic city landscape, neon, digital art")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="ugly, blurry, poor detail")
            background_mode = gr.Radio(["AI Generation", "Custom Upload"], value="AI Generation", label="Background Mode")
            user_bg = gr.Image(label="Upload Background", visible=False)
            blend_mode = gr.Dropdown(["Overlay", "Multiply", "Alpha"], value="Overlay", label="Blend Mode")
            guidance_scale = gr.Slider(1, 20, value=8.0, label="Prompt Strength")
            controlnet_scale = gr.Slider(0.5, 2.5, value=1.5, label="QR Code (Scannability) Strength")
            num_steps = gr.Slider(10, 50, value=30, label="Quality Steps")
            submit = gr.Button("Generate Art QR")
        with gr.Column():
            output_img = gr.Image(label="Generated QR Code", interactive=False)
            scan_status = gr.Textbox(label="Scan Status", interactive=False)
    def update_bg_visibility(mode):
        return gr.update(visible=(mode == "Custom Upload"))
    background_mode.change(fn=update_bg_visibility, inputs=background_mode, outputs=user_bg)
    submit.click(
        qr_stylizer,
        inputs=[data, prompt, negative_prompt, background_mode, user_bg, blend_mode, guidance_scale, controlnet_scale, num_steps],
        outputs=[output_img, scan_status]
    )

demo.launch(share=True, debug=True)


ModuleNotFoundError: No module named 'qrcode'

In [2]:
# Install required packages
!pip install qrcode[pil] pyzbar gradio diffusers transformers scipy ftfy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
import torch
import qrcode
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from PIL import Image, ImageOps, ImageEnhance
import gradio as gr
import io
from pyzbar.pyzbar import decode

# --- QR Generation Module ---
def generate_qr(data, size=512):
    qr = qrcode.QRCode(
        version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=10, border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    return img.resize((size, size))

# --- Blending Module ---
def blend_images(qr_img, bg_img, mode):
    qr_mask = qr_img.convert("L").point(lambda x: 0 if x < 128 else 255, '1')
    bg_img = bg_img.resize(qr_img.size)
    if mode == "Overlay":
        composite = Image.composite(qr_img, bg_img, qr_mask)
    elif mode == "Multiply":
        composite = Image.blend(bg_img, qr_img, alpha=0.5)
    else:
        composite = Image.alpha_composite(bg_img.convert("RGBA"), qr_img.convert("RGBA"))
    return composite.convert("RGB")

# --- AI Model Setup ---
controlnet = ControlNetModel.from_pretrained(
    "monster-labs/control_v1p_sd15_qrcode_monster",
    torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# --- Scannability Check Module ---
def is_qr_scannable(image):
    image = image.convert("RGB")
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")
    buffer.seek(0)
    result = decode(Image.open(buffer))
    return len(result) > 0

# --- Main Pipeline ---
def qr_stylizer(
    data, prompt, negative_prompt, background_mode, user_bg, blend_mode,
    guidance_scale, controlnet_scale, num_steps
):
    qr_img = generate_qr(data)

    # Background selection
    if background_mode == "AI Generation":
        base_img = qr_img
    else:
        if user_bg is None:
            return None, "Please upload a background image."
        bg_img = Image.open(user_bg).convert("RGB")
        qr_blended = blend_images(qr_img, bg_img, blend_mode)
        base_img = qr_blended

    generated = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=base_img,
        guidance_scale=float(guidance_scale),
        controlnet_conditioning_scale=float(controlnet_scale),
        num_inference_steps=int(num_steps),
    ).images[0]

    # Validate QR
    scan_ok = is_qr_scannable(generated)
    if not scan_ok:
        msg = "⚠️ Warning: QR code may not be scannable! Try decreasing style intensity or adjusting parameters."
    else:
        msg = "✅ Success: QR code is scannable."

    return generated, msg

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("## 🎨 Artistic, Scannable QR Code Generator")
    with gr.Row():
        with gr.Column():
            data = gr.Textbox(label="Data (URL/Text)", value="https://google.com")
            prompt = gr.Textbox(label="Prompt (For AI Generation)", value="Futuristic city landscape, neon, digital art")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="ugly, blurry, poor detail")
            background_mode = gr.Radio(["AI Generation", "Custom Upload"], value="AI Generation", label="Background Mode")
            user_bg = gr.Image(label="Upload Background", visible=False)
            blend_mode = gr.Dropdown(["Overlay", "Multiply", "Alpha"], value="Overlay", label="Blend Mode")
            guidance_scale = gr.Slider(1, 20, value=8.0, label="Prompt Strength")
            controlnet_scale = gr.Slider(0.5, 2.5, value=1.5, label="QR Code (Scannability) Strength")
            num_steps = gr.Slider(10, 50, value=30, label="Quality Steps")
            submit = gr.Button("Generate Art QR")
        with gr.Column():
            output_img = gr.Image(label="Generated QR Code", interactive=False)
            scan_status = gr.Textbox(label="Scan Status", interactive=False)
    def update_bg_visibility(mode):
        return gr.update(visible=(mode == "Custom Upload"))
    background_mode.change(fn=update_bg_visibility, inputs=background_mode, outputs=user_bg)
    submit.click(
        qr_stylizer,
        inputs=[data, prompt, negative_prompt, background_mode, user_bg, blend_mode, guidance_scale, controlnet_scale, num_steps],
        outputs=[output_img, scan_status]
    )

demo.launch(share=True, debug=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

In [4]:
!apt-get update
!apt-get install -y libzbar0


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,705 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/univers

In [5]:
!pip install pyzbar
